## Final

In [1]:
import pandas as pd
from datetime import datetime
import pytz

# sqoff_performance sheet 1 
sqoff_df = pd.read_excel(r"C:\Users\USER\Downloads\sqoff_performance (9).xlsx", dtype={'user_id': str})

# Clean and standardize key columns
sqoff_df['user_id'] = sqoff_df['user_id'].astype(str).str.strip()
sqoff_df['order_side'] = sqoff_df['order_side'].astype(str).str.strip().str.upper()
sqoff_df['order_status'] = sqoff_df['order_status'].astype(str).str.strip().str.title()

# Convert order_fire_time to datetime and keep original string version
sqoff_df['order_fire_time'] = pd.to_datetime(sqoff_df['order_fire_time'], format='%H:%M:%S.%f', errors='coerce')

# Find minimum and maximum order_fire_time per user_id
min_max_order_times = (
    sqoff_df.groupby('user_id')['order_fire_time']
    .agg(['min', 'max'])
    .dropna()
    .reset_index()
)

min_max_order_times['min_order_fire_time'] = min_max_order_times['min'].dt.strftime('%H:%M:%S.%f').str[:-3]
min_max_order_times['max_order_fire_time'] = min_max_order_times['max'].dt.strftime('%H:%M:%S.%f').str[:-3]
min_order_times = min_max_order_times[['user_id', 'min_order_fire_time', 'max_order_fire_time']]

# Performance calculation
results = []
for user_id, group in sqoff_df.groupby('user_id'):
    buy_orders = group[group['order_side'] == 'BUY']
    total_buy = len(buy_orders)
    completed_buy = len(buy_orders[buy_orders['order_status'].isin(['Filled', 'Complete'])])
    rejected_buy = len(buy_orders[buy_orders['order_status'] == 'Rejected'])
    performance_ratio = completed_buy / total_buy if total_buy > 0 else 0
    critical_exchange = len(buy_orders[
        (buy_orders['order_status'].isin(['Filled', 'Complete'])) & 
        (buy_orders['exchange_response_ms'] > 1000)
    ])
    critical_order = len(buy_orders[
        (buy_orders['order_status'].isin(['Filled', 'Complete'])) & 
        (buy_orders['order_response_ms'] > 1200)
    ])
    results.append({
        'user_id': user_id,
        'total_buy_orders': total_buy,
        'completed_buy_orders': completed_buy,
        'rejected_buy_orders': rejected_buy,
        'performance_ratio': performance_ratio,
        'critical_exchange_ms (>1000ms)': critical_exchange,
        'critical_order_ms (>1200ms)': critical_order
    })

new_df = pd.DataFrame(results)

# User info
users_df = pd.read_excel(r"C:\Users\USER\Downloads\sqoff_performance (9).xlsx", sheet_name='users', dtype={'userId': str})
users_df.rename(columns={'userId': 'user_id'}, inplace=True)
users_df['sl_broker'] = users_df['max_loss'] / users_df['broker_sl']
users_df['should_be_rms'] = users_df['sl_broker'].apply(lambda x: 'MSTECH' if x < 1.09 else 'RMS')
users_df['sqoff_initiated_time'] = pd.to_datetime(
    users_df['sqoff_initiated_time'], errors='coerce'
).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata').dt.strftime('%Y-%m-%d %H:%M:%S.%f')

# MTM info
mtm_df = pd.read_excel(r"C:\Users\USER\Downloads\saved_mtm_2025-07-08 (1).xlsx", dtype={'user_id': str})
mtm_df['MTM%'] = (mtm_df['MTM'] / mtm_df['allocation'] * 100).round(3)

# Stoploss sent file
stoploss_df_path = r"C:\Users\USER\Downloads\STOPLOSS SENT-data-2025-07-08 10_29_08.csv"
df_raw = pd.read_csv(stoploss_df_path, header=None)

rows = []
for _, row in df_raw.iterrows():
    timestamp_str = str(row[0]).strip()
    user_data = str(row[1]).strip()
    if timestamp_str.isdigit():
        dt = datetime.fromtimestamp(int(timestamp_str) / 1000.0, tz=pytz.UTC)
        timestamp = dt.astimezone(pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        user_ids = [uid.strip() for uid in user_data.split(',') if uid.strip()]
        for uid in user_ids:
            rows.append([timestamp, uid])

df_melted = pd.DataFrame(rows, columns=["stoploss_sent", "user_id"])
df_melted["user_id"] = df_melted["user_id"].astype(str)

# Merge everything
merged_result = (
    mtm_df
    .merge(new_df, on='user_id', how='left')
    .merge(users_df, on='user_id', how='left')
    .merge(df_melted, on='user_id', how='left')
    .merge(min_order_times, on='user_id', how='left')
)

# Flag if stoploss was sent
merged_result['triggered_stoploss_sent'] = ~merged_result['stoploss_sent'].isna()

# Convert timestamps
merged_result['stoploss_sent_dt'] = pd.to_datetime(merged_result['stoploss_sent'], errors='coerce')
merged_result['sqoff_initiated_time_dt'] = pd.to_datetime(merged_result['sqoff_initiated_time'], errors='coerce')

# Time diff in milliseconds
merged_result['sqoff_stoploss_diff_ms'] = (
    (merged_result['stoploss_sent_dt'] - merged_result['sqoff_initiated_time_dt'])
    .dt.total_seconds() * 1000
)

# Critical condition
merged_result['critical'] = (
    ((merged_result['sqoff_stoploss_diff_ms'] > 200)) |
    (merged_result['MTM%'] < -1.3) |
    ((merged_result['critical_order_ms (>1200ms)'].fillna(0) > 0) & 
     (merged_result['critical_exchange_ms (>1000ms)'].fillna(0) > 0)) |
    (merged_result['total_buy_orders'].fillna(0) == 0) |
    (merged_result['completed_buy_orders'].fillna(0) == 0) |
    (merged_result['performance_ratio'].fillna(1) < 0.10) |
    (merged_result['triggered_stoploss_sent'] != True)
)

merged_result['critical'] = merged_result['critical'].fillna(False)

# Final selected columns
final_columns = [
    'critical', 'user_id', 'alias', 'broker', 'MTM', 'MTM%', 'algo_x', 'server',
    'allocation_x', 'capital_x', 'max_loss_x', 'total_buy_orders', 'completed_buy_orders',
    'rejected_buy_orders', 'performance_ratio', 'critical_exchange_ms (>1000ms)',
    'critical_order_ms (>1200ms)', 'sqoff_initiated', 'sqoff_initiated_time', 'broker_sl',
    'should_be_rms', 'sl_broker', 'stoploss_sent', 'min_order_fire_time','max_order_fire_time',
    'triggered_stoploss_sent', 'sqoff_stoploss_diff_ms', 'org', 'realizedMTM',
    'unRealizedMTM', 'date'
]

# Filter the DataFrame
final_df = merged_result[final_columns]

# Export to Excel or CSV
# final_df.to_excel("final_sqoff_analysis.xlsx", index=False)  # or .to_csv("final_sqoff_analysis.csv", index=False)



In [2]:
final_df.head(5)

,critical,user_id,alias,broker,MTM,MTM%,algo_x,server,allocation_x,capital_x,...,sl_broker,stoploss_sent,min_order_fire_time,max_order_fire_time,triggered_stoploss_sent,sqoff_stoploss_diff_ms,org,realizedMTM,unRealizedMTM,date
0,True,MELOGIST,MSR_ELOGIST,IIFL_2,258461.99,1.292,8,VS1,20000000,31735329.0,...,-0.541419,NaN,NaN,NaN,False,NaN,megaserve,-1499.380000,0.00,2025-07-08
1,True,MRUTVI02,MSR_RUTVICH,IIFL_2,255470.00,1.277,8,VS1,20000000,45491448.0,...,-0.377197,NaN,NaN,NaN,False,NaN,megaserve,-3519.653757,0.00,2025-07-08
2,True,DPNOGS01,CC_SISL_GS,SHARE_INDIA,233688.01,0.779,7,VS8,30000000,51000000.0,...,-0.653595,NaN,NaN,NaN,False,NaN,megaserve,-346.250000,222551.25,2025-07-08
3,True,AJIT0112,MSR_F14_AJITAG,IIFL,258949.00,1.295,8,VS1,20000000,30393113.0,...,-0.565358,NaN,NaN,NaN,False,NaN,megaserve,-1544.740000,0.00,2025-07-08
4,True,SWAASTIK,MSR_SWASTIK_INVESTMENTS,IIFL,259246.00,1.296,8,VS1,20000000,31545687.0,...,-0.544746,NaN,NaN,NaN,False,NaN,megaserve,-1506.500000,0.00,2025-07-08


In [3]:
output = final_df.to_excel(r"C:\Users\USER\Downloads\26-06-2025_Sqoff_analysis.xlsx", index=False)

In [20]:
from datetime import datetime
import pytz

timestamp_ms = 1749528404182

dt = datetime.fromtimestamp(timestamp_ms / 1000.0, tz=pytz.UTC)
dt_india = dt.astimezone(pytz.timezone('Asia/Kolkata'))
print(dt_india.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])


2025-06-10 09:36:44.182
